<a href="https://colab.research.google.com/github/aztofighi/LLM-Essay-Creator/blob/main/Copy_of_Create_AI_generated_essays_using_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate

In [ ]:
!condacolab KERNEL RESTART
print("Restarting of kernel...")
get_ipython().kernel.do_shutdown(True)

In [ ]:
import accelerate

In [ ]:
!pip install faker --quiet

In [ ]:
import torch
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {DEVICE}")
print(f"CUDA Version: {torch.version.cuda}")
print(f"Pytorch {torch.__version__}")

Device: cuda
CUDA Version: 12.1
Pytorch 2.2.1+cu121


In [ ]:
import sys, random, string, re, time, os
import pandas as pd
import numpy as np
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from tqdm.auto import tqdm
from faker import Faker  #generates fake data
from spacy.lang.en import English

In [ ]:
import torch, random
# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

SEED = 42
# Seed the same seed to all
def seed_everything(seed=42):
    Faker.seed(0)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything(SEED)

In [ ]:
import ctypes, gc, torch
libc = ctypes.CDLL("libc.so.6")
def clear_memory():
    libc.malloc_trim(0)
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
# Random generate 12 random number
def get_userid(length=16):
    """Generate userid - """
    userid = str(int(np.random.rand()*1_000_000_000))
    # Add the extra rand chars
    for i in range(length):
        # Select random char or digital number (0-9)
        userid = userid + random.choice(string.ascii_letters + str(random.randint(0, 9)))
    return userid

# Generate the personal url from social media
def generate_fake_social_media_url(user_name):
    social_media_platforms = {
        'LinkedIn': 'linkedin.com/in/',
        'YouTube': 'youtube.com/c/',
        'Instagram': 'instagram.com/',
        'GitHub': 'github.com/',
        'Facebook': 'facebook.com/',
        'Twitter': 'twitter.com/'
    }
    platform, domain = random.choice(list(social_media_platforms.items()))
    fake_url = f'https://{domain}{user_name}'
    return fake_url

def generate_username(first_name, last_name, fake_user_name):
    """usernames are created from first_name and last_name"""
    SEPS = ["_", ".", ""]
    if random.choice([False, True]):
        username = f"{first_name.lower()}{random.choice(SEPS)}{last_name.lower()}{random.randint(1,999)}"
    else:
        username = fake_user_name
    return username

def generate_email(first_name, last_name, faker):
    """usernames are created from first_name and last_name"""
    SEPS = ["_", ".", ""]
    if random.choice([False, True]):
        email = f"{first_name.lower()}{random.choice(SEPS)}{last_name.lower()}@{faker.domain_name()}"
    else:
        email = faker.email()
    return email

def generate_student_info():
    """Generates all the user info (name, eamil addresses, phone number, etc) together """
    # Select the student country to generate the user info based on the country
    COUNTRIES = ["en_US", "en_US", "en_US", "en_US", "en_US", "en_US",
                 "de_DE", "it_IT", "es_ES", "da_DK", "fr_FR", "vi_VN"]
    country = random.choice(COUNTRIES)
    faker = Faker(country)
    first_name = faker.first_name()
    last_name = faker.last_name()
    user_name = generate_username(first_name, last_name, faker.user_name())
    fake_url = generate_fake_social_media_url(user_name)
    student = {}
    student['COUNTRY'] = country
    student['ID_NUM'] = get_userid() # User ID
    student['NAME_STUDENT'] = first_name + " "+  last_name
    student['EMAIL'] = generate_email(first_name, last_name, faker)
    student['USERNAME'] = user_name
    student['PHONE_NUM'] = faker.phone_number().replace(" ", "")
    student['URL_PERSONAL'] = fake_url
    student['STREET_ADDRESS'] = str(faker.address()).replace("\n"," ") # Replace \n with space in the address
    del faker
    clear_memory()
#     print(student)
    return student

In [ ]:
from pathlib import Path
# Copy the generated df to working folder
import shutil

GENERATED = True
# Create the folder
Path("/content").mkdir(parents=True, exist_ok=True)
if GENERATED:
    TOTAL = 100 # Generate 100
    students = []
    for i in tqdm(range(TOTAL)):
        students.append(generate_student_info())
        print(f"Generate {i}-th information")
    df = pd.DataFrame(students)
    df = df.reset_index(drop=True)
    # Save to the csv file
    df.to_csv("/content/df.csv", index=False, encoding='UTF-8') # Do not save default ID column
    display(df.tail(10))
    # Check if ID_NUM has any duplicates
    assert df['ID_NUM'].duplicated().value_counts()[False] == TOTAL, "Duplicated ID_NUM"
else:
    shutil.copy('/kaggle/input/ai-generated-text-dataset/temp/df.csv', '/kaggle/working/temp/df.csv')

  0%|          | 0/100 [00:00<?, ?it/s]

Generate 0-th information
Generate 1-th information
Generate 2-th information
Generate 3-th information
Generate 4-th information
Generate 5-th information
Generate 6-th information
Generate 7-th information
Generate 8-th information
Generate 9-th information
Generate 10-th information
Generate 11-th information
Generate 12-th information
Generate 13-th information
Generate 14-th information
Generate 15-th information
Generate 16-th information
Generate 17-th information
Generate 18-th information
Generate 19-th information
Generate 20-th information
Generate 21-th information
Generate 22-th information
Generate 23-th information
Generate 24-th information
Generate 25-th information
Generate 26-th information
Generate 27-th information
Generate 28-th information
Generate 29-th information
Generate 30-th information
Generate 31-th information
Generate 32-th information
Generate 33-th information
Generate 34-th information
Generate 35-th information
Generate 36-th information
Generate 37

,COUNTRY,ID_NUM,NAME_STUDENT,EMAIL,USERNAME,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS
90,en_US,119594245AiylOSJFiuNwFmJP,Melvin Rowe,melvin.rowe@day-watson.org,natashamccullough,001-443-201-7823x695,https://facebook.com/natashamccullough,"5039 Carlos Curve Apt. 886 Jaredchester, AK 71876"
91,fr_FR,713244787kKYsrIkKJhhX8IWI,Christiane Wagner,veronique67@example.net,christiane.wagner728,0183015666,https://github.com/christiane.wagner728,"96, avenue Bousquet 36789 Brunetnec"
92,de_DE,760785048tCwCXnSTOki3ZKIM,Gundula Losekann,walter85@example.com,xtrueb,0925067741,https://twitter.com/xtrueb,Pierre-Barth-Weg 2 71129 Geithain
93,en_US,561277197RidPSpTpFKOyezVS,Deborah White,deborah.white@sanders-collins.net,deborah.white587,6278211744,https://instagram.com/deborah.white587,"7104 Sarah Motorway Suite 836 South Billyland,..."
94,it_IT,770967179UOezBFuvKKYRcObk,Lando Mascheroni,lando.mascheroni@battaglia.com,landomascheroni3,+39012582111,https://twitter.com/landomascheroni3,"Strada Carosone, 7 Appartamento 95 34076, Mede..."
95,en_US,493795596kfqxdOUBTaEioRaS,Courtney Mullen,courtney_mullen@kemp-rice.com,courtney_mullen706,+1-898-764-3912,https://youtube.com/c/courtney_mullen706,"500 Douglas Streets Apt. 221 Navarroberg, KY 3..."
96,de_DE,522732829ofgxiXPeKNCUOARn,Margot Etzold,margareta53@example.net,gerlachsina,+49(0)8724531935,https://github.com/gerlachsina,Wohlgemutstraße 0 36068 Ebern
97,en_US,427541018UwupPpMUsQnMmsYF,Samantha Williams,samantha_williams@ayala.com,jennifer56,+1-824-658-1273x9991,https://linkedin.com/in/jennifer56,"0092 Jackson Lodge New Amy, ID 04639"
98,en_US,25419126auyAIWXbeqcYGOXB,Dawn Mullins,dawn_mullins@watkins-nixon.com,dawnmullins609,(910)865-2736x903,https://linkedin.com/in/dawnmullins609,"66734 Jacqueline Estate West Rhondaburgh, CT 2..."
99,fr_FR,107891426tGZIzOtUdfhNdQhT,Philippe Schmitt,ilebrun@example.net,hboucher,0767149544,https://instagram.com/hboucher,"880, boulevard de Gaudin 25532 Foucher"


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.model_download("mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf")

print("Path to model files:", path)

100%|██████████| 11.1G/11.1G [01:57<00:00, 102MB/s]
Extracting model files...


Path to model files: /root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1


In [ ]:
def load_model():
    torch.backends.cuda.enable_mem_efficient_sdp(False)
    torch.backends.cuda.enable_flash_sdp(False)
    model_path="/root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1"
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

    model = AutoModelForCausalLM.from_pretrained(model_path,
                                                 torch_dtype=torch.bfloat16,
                                                 device_map="auto")
    return model, tokenizer

In [ ]:
def generate_on_user_info(row):
    # Prompt template generates the texts
    prompt_template = """<s>[INST]
    You are an essay writer.
    You will be given some personal information about the student writing it, like
    name = {first_name} {last_name}
    email = {email}
    street address = {address}
    phone number = {phone_num}
    personal url = {url}
    username = {username}
    user id = {userid}
    You need to write a short essay that includes all the given information somewhere in the essay.
    Do not miss out any.[/INST]"""

    first_name = row['NAME_STUDENT'].split()[0]
    last_name = row['NAME_STUDENT'].split()[1]
    email = row['EMAIL']
    phone_num = row['PHONE_NUM']
    address = row['STREET_ADDRESS']
    url = row['URL_PERSONAL']
    username = row['USERNAME']
    userid = row['ID_NUM']
    # Fill in prompt with PII
    prompt = prompt_template.format(first_name=first_name,
                                    last_name=last_name,
                                    email=email,
                                    phone_num=phone_num,
                                    address=address,
                                    url=url,
                                    username=username,
                                    userid=userid
                                   )
    return prompt

def generate_texts(model, tokenizer, df, num_essays):
    generated_df = df[:num_essays]
    # Generate the texts
    for i in tqdm(range(len(generated_df))):
        start = time.time()
        # Generate PII
        row = generated_df.loc[i]
        # Generate the texts using three prompts
        prompt = generate_on_user_info(row)
        # Tokenize the prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

        # Generate the outputs from prompt
        generate_ids = model.generate(**inputs,
                                      max_new_tokens=768,
                                      do_sample=True,
                                      temperature=0.7,
                                      top_p=0.95,
                                      top_k=40,
                                      repetition_penalty=1.1,
                                      pad_token_id=tokenizer.eos_token_id
                                     )
        # Decode the generated output
        generated_text = tokenizer.batch_decode(generate_ids, skip_special_tokens=True,
                                                 clean_up_tokenization_spaces=False)[0]
        generated_text = generated_text.split('[/INST] ')[1]
#         print(f"generated_text = {generated_text}" )
        generated_df.loc[i, 'generated_text'] = generated_text
        clear_memory()
        print(f"Complete the text for {i}-th student {time.time() - start: .1f} seconds")
    # Save generated_df to csv
    generated_df.to_csv("temp/generated_df.csv", index=False, encoding="UTF-8")

In [ ]:
# Load the generated df from Gemma
df = pd.read_csv('/kaggle/input/text-generation-gemma/generated_df_0_100.csv', encoding='UTF-8')
df2 = pd.read_csv('/kaggle/input/text-generation-gemma/generated_df_100_900.csv', encoding='UTF-8')
df3 = pd.read_csv('/kaggle/input/text-generation-gemma/generated_df_900_1700.csv', encoding='UTF-8')
generated_df = pd.concat([df, df2, df3])
generated_df = generated_df.reset_index(drop=True, inplace=False)
generated_df = generated_df.rename({'generated_text': 'Essay'}, axis=1)
# Load the generated df from Gemini
df4 = pd.read_csv('/kaggle/input/pii-detection-gemini-created-dataset/pii_gemini.csv',
                  encoding='UTF-8', index_col=[0]) # Load df without unnamed columns
df4['NAME_STUDENT'] = ['' for _ in range(len(df4))] # The dataset lacks of student name
df4 = df4.reset_index(drop=True, inplace=False)
# Combine both datasets
generated_df = pd.concat([generated_df, df4])
# generated_df = df4
generated_df = generated_df.reset_index(drop=True, inplace=False)
display(generated_df.tail(1))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/text-generation-gemma/generated_df_0_100.csv'

In [ ]:
GENERATED = True  # True: Generate texts
if GENERATED:
    model, tokenizer = load_model()
    generate_texts(model, tokenizer, df, num_essays=20)
    sys.exit(0)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

<ipython-input-11-bb0c0d9b7130>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  generated_df.loc[i, 'generated_text'] = generated_text


Complete the text for 0-th student  81.3 seconds
Complete the text for 1-th student  74.5 seconds
Complete the text for 2-th student  27.2 seconds
Complete the text for 3-th student  84.6 seconds
Complete the text for 4-th student  136.9 seconds
Complete the text for 5-th student  72.0 seconds
Complete the text for 6-th student  140.5 seconds
Complete the text for 7-th student  59.4 seconds
Complete the text for 8-th student  153.6 seconds
Complete the text for 9-th student  93.4 seconds
Complete the text for 10-th student  80.2 seconds
Complete the text for 11-th student  62.4 seconds
Complete the text for 12-th student  150.0 seconds
Complete the text for 13-th student  141.5 seconds
Complete the text for 14-th student  107.0 seconds
Complete the text for 15-th student  110.5 seconds
Complete the text for 16-th student  17.9 seconds
Complete the text for 17-th student  19.8 seconds
Complete the text for 18-th student  104.0 seconds
Complete the text for 19-th student  97.8 seconds


OSError: Cannot save file into a non-existent directory: 'temp'

In [ ]:
label_types = ['NAME_STUDENT','EMAIL', 'USERNAME', 'ID_NUM',
               'PHONE_NUM', 'URL_PERSONAL', 'STREET_ADDRESS']

In [ ]:
from spacy.lang.en import English
import re

en_tokenizer = English().tokenizer

def tokenize_with_spacy(text, tokenizer=en_tokenizer):
    tokenized_text = tokenizer(text)
    tokens = [token.text for token in tokenized_text]
    trailing_whitespace = [bool(token.whitespace_) for token in tokenized_text]
    return tokens, trailing_whitespace

# Update labels and boolean flags
def update_labels(i, token, label_type, labels, isFirst_flags):
#     print(f"Found {i}-th position token: {token}")
    # Update the label
    if isFirst_flags[label_type]:
        labels[i] = 'B-'+label_type # Beginning of an entity
        isFirst_flags[label_type] = False
    else:
        labels[i] = 'I-'+label_type # Contiunity of an entity
    return labels, isFirst_flags

# Go through each token and assign name label ('NAME_STUDENT') if matched
def assign_name(names, tokens, labels):
    #print(f"Search 'NAME_STUDENT': {names}")
    for i, token in enumerate(tokens):
        token = str(token).lower()
        # Order does not matter
        if token in names:
            # If the previous
            if i > 0 and labels[i-1] == 'B-NAME_STUDENT':
                labels[i] = 'I-NAME_STUDENT'
            else:
                labels[i] = 'B-NAME_STUDENT'
    return labels

def assign_phone_number(phones, tokens, labels, isFirst_flags):
#     print(f"Search 'PHONE_NUM': {phone_tokens}")
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        label = labels[i]
        # Add a special case '-'
        if token == '-':
            # Check if the previous token is not 'O'
            if labels[i-1] != 'O':
                # Update the labels
                labels, isFirst_flags = update_labels(i, token_, 'PHONE_NUM',
                                                      labels, isFirst_flags)
        else:
            # Check if token is matched last name
            if label == 'O' and token_ in phones:
                labels, isFirst_flags = update_labels(i, token_, 'PHONE_NUM',
                                                      labels, isFirst_flags)
    return labels, isFirst_flags

# Go through each token and assign street address label
def assign_street_address(address, tokens, labels):
    # print(tokens)
    # Keep track of index for a long label
    label_index = 0
    reserve_indices = []
    sandwich_max_size = 3
    #print(f'address = {address}')
    for i, token in enumerate(tokens):
        try:
            #Order matters and sandwiches are possible
            token = str(token).lower()
            curr_idx = label_index
            curr_token = address[curr_idx]
            if token == curr_token:
                # print(f"token = {token}")
                #case where a token corresponds to the expected next token
                if len(reserve_indices) > sandwich_max_size:
                    reserve_indices = []

                prefix = 'B-' if curr_idx == 0 else 'I-'
                labels[i] = prefix + 'STREET_ADDRESS'
                #fill sandwiches if the next token has been found
                for k in reserve_indices:
                    labels[k] = 'I-STREET_ADDRESS'
                reserve_indices = []
                #Update positional pointer
                label_index += 1
                # At the end of address
                if label_index == len(address):
                    label_index = 0
                # print(f'label_index = {label_index}')
            else:
                reserve_indices.append(i)
                #print(f"! {token}")
        except Exception as e:
            print(f"Error occurs at {i}-th {token} \n{e}" )
            sys.exit(-1)
#         #case where some surprise token has been added in the PII
#     address_labels = [(label, token) for i, (label, token) in enumerate(zip(labels, tokens))
#                       if label =='B-STREET_ADDRESS' or label == 'I-STREET_ADDRESS']
#     print(address_labels)

    return labels

# Assign labels for other types
def assign_other_label_types(essay, label_type, tokens, labels, isFirst_flags):
    label_value = essay[label_type].lower()
#     print(f"Search '{label_type}': {label_value}")
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        # Check if token is first or last name
        if label_value in token_:
            # Update the label
            labels, isFirst_flags = update_labels(i, token_, label_type,
                                                      labels, isFirst_flags)
    return labels, isFirst_flags

# Assign labels for other types
def assign_email(email, tokens, labels):
    #print(f"Search Email: {email}")
    is_First = False
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        # Check if token is first or last name
        if email in token_:
            # print(f'Token {token_}')
            if not is_First:
                # Update the label
                labels[i] = 'B-EMAIL'
                is_First = True
            else: # Skip labeling
                print(f"Duplicated Email {email}")
    return labels

# Assign labels for other types
def assign_username(username, tokens, labels):
    #print(f"Search username: {username}")
    is_First = False
    for i, token in enumerate(tokens):
        token_ = str(token).lower()
        # Check if token is first or last name
        if username in token_:
            # print(f'Token {token_}')
            if not is_First:
                # Update the label
                labels[i] = 'B-USERNAME'
                is_First = True
            else: # Skip labeling
                print(f"Duplicated username {username}")
    return labels

In [ ]:
# “B-”: the beginning of an entity.
# “I-”: the next of an entity
def assign_labels(essay, tokens):
    # print(f"essay['NAME_STUDENT'] = {essay['NAME_STUDENT']}" )
    # Assign "O" to labels by default
    labels = ['O' for token in tokens]
    # Create a boolean flag list to track if a label type start the text.
    isFirst_flags = {label_type: True for label_type in label_types}
    # Go through each token and check if the label appear in the token
    # All token and label values are lower case for comparison
    for label_type in label_types:
        if label_type == 'NAME_STUDENT':
            if essay['NAME_STUDENT'] != '':
                names, _ = tokenize_with_spacy(essay['NAME_STUDENT'].lower())
                labels = assign_name(names, tokens, labels)
        elif label_type == 'STREET_ADDRESS':
            address = essay['STREET_ADDRESS'].lower().replace("\\n", " ")
            # print(f"address {address}")
            address = address.translate(str.maketrans('', '', string.punctuation)) # Remove punctuations
            address = address.split(' ')
            if len(address) > 0:
                labels = assign_street_address(address, tokens, labels)
        elif label_type == 'PHONE_NUM':
            phones, _ = tokenize_with_spacy(essay['PHONE_NUM'].lower())
            labels, isFirst_flags = assign_phone_number(phones, tokens, labels, isFirst_flags)
        elif label_type == 'EMAIL':
            email = essay['EMAIL'].lower()
            assign_email(email, tokens, labels)
        elif label_type == 'USERNAME':
            username = essay['USERNAME'].lower()
            assign_username(username, tokens, labels)
        else:
            labels, isFirst_flags = assign_other_label_types(essay, label_type, tokens,
                                                             labels, isFirst_flags)
    return labels

In [ ]:
def process_full_text(full_text):
    if 'Unprocessed:' in full_text: # Need to split the instruction and response
        pattern = r'([*]*Please write an essay about [^*]*[*]*)'
        x = re.search(pattern, full_text)
        if x:
            splits = re.split(pattern, full_text)
            text = splits[-1].strip()
            # print(f"=== Split text ===\n {text}")
            return text
        else:
            print(f"### UnProcess text###\n {full_text}")
            return None
    else:
        return full_text.strip() # Remove the space to the left

# Map the label to token
def create_token_map(tokens, labels):
    token_map = []
    for i, label in enumerate(labels):
        if label != 'O':
            token_map.append({label: (tokens[i], i)})
    return token_map

In [ ]:
# Assign the labels to the tokens
results = []
doc_id = 1_221_555_000 # document id
for i in range(len(generated_df)):
    row = generated_df.iloc[i]
    full_text = process_full_text(row["Essay"])
    if full_text:
        # Tokenize the text using spacy tokenizer
        tokens, trailing_whitespace = tokenize_with_spacy(full_text)
        labels = assign_labels(row, tokens)
        token_map = create_token_map(tokens, labels)
        doc_id += 1
        result = {
            'document': doc_id,
            'full_text': full_text,
            'tokens': tokens,
            'trailing_whitespace': trailing_whitespace,
            'labels': labels,
            'token_map': token_map
        }
        # Add PII to result
        for label_type in label_types:
            result[label_type] = row[label_type]
        # print(result)
        results.append(result)
# # Save to temp fold for verification
df = pd.DataFrame(results)
df.to_csv("temp/pii_dataset_Gemma.csv", index=False, encoding="UTF-8")
display(df)

NameError: name 'generated_df' is not defined

In [ ]:
# A function displays PII (like phone number) to help double-check the generated text
def verify_df(df):
    tmp = df
    tmp = tmp.reset_index(drop=True)
    for i in range(len(tmp)):
        row = tmp.iloc[i]
        full_text = row['full_text']
        tokens = row['tokens']
        token_map = row['token_map']
        address = row['STREET_ADDRESS']
        # Display full text and all
        print(f"=== Doc {i} ===\n"
              f"full_text = {full_text}\n"
              f"address = {address}")
        for t_dic in token_map:
            if 'B-STREET_ADDRESS' in t_dic.keys() or 'I-STREET_ADDRESS' in t_dic.keys():
                print(t_dic)
tmp = df[0:10]
verify_df(tmp)